# BIF - Projet Minimapper

#### Objectifs
1. FM-index et la BWT pour indexer le génome de référence (TP 4)  
2. Pattern matching exacte de taille k (ancre)  
3. Programme de programmation dynamique implémenté dans le TP 5

#### Bonus
4. Aligner rev-complement
5. Améliorer le temps de calcul
6. Améliorer l’empreinte mémoire

## Get BWT (Tests)

In [1]:
from indexReference import IndexReference

In [12]:
index = IndexReference("AGTCGTC$")

In [13]:
index

In [14]:
index.sa

[7, 0, 6, 3, 4, 1, 5, 2]

In [15]:
index.bwt

['C', '$', 'T', 'T', 'C', 'A', 'G', 'G']